In [7]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import json
import os
from decimal import Decimal
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from collections import defaultdict

class TransacaoFinanceira:
    def __init__(self, descricao, valor, categoria, tipo, data=None):
        self.descricao = descricao
        self.valor = Decimal(str(valor))
        self.categoria = categoria
        self.tipo = tipo
        self.data = data if data else datetime.now().strftime("%d/%m/%Y")

    def to_dict(self):
        return {
            "descricao": self.descricao,
            "valor": str(self.valor),
            "categoria": self.categoria,
            "tipo": self.tipo,
            "data": self.data
        }

    @classmethod
    def from_dict(cls, data):
        return cls(
            data["descricao"],
            data["valor"],
            data["categoria"],
            data["tipo"],
            data["data"]
        )

class SistemaFinancas:
    def __init__(self):
        self.transacoes = []
        self.arquivo_dados = "financas.json"
        self.categorias = [
            "Alimentação",
            "Transporte",
            "Moradia",
            "Lazer",
            "Saúde",
            "Educação",
            "Outros"
        ]
        self.carregar_dados()

    def adicionar_transacao(self, transacao):
        self.transacoes.append(transacao)
        self.salvar_dados()

    def remover_transacao(self, indice):
        if 0 <= indice < len(self.transacoes):
            del self.transacoes[indice]
            self.salvar_dados()

    def calcular_saldo(self):
        saldo = Decimal('0')
        for transacao in self.transacoes:
            if transacao.tipo == "Receita":
                saldo += transacao.valor
            else:
                saldo -= transacao.valor
        return saldo

    def salvar_dados(self):
        dados = {
            "transacoes": [t.to_dict() for t in self.transacoes]
        }
        with open(self.arquivo_dados, "w", encoding="utf-8") as arquivo:
            json.dump(dados, arquivo, ensure_ascii=False, indent=2)

    def carregar_dados(self):
        if os.path.exists(self.arquivo_dados):
            try:
                with open(self.arquivo_dados, "r", encoding="utf-8") as arquivo:
                    dados = json.load(arquivo)
                    self.transacoes = [
                        TransacaoFinanceira.from_dict(t) for t in dados["transacoes"]
                    ]
            except:
                self.transacoes = []

class InterfaceGrafica:
    def __init__(self, root):
        self.root = root
        self.sistema = SistemaFinancas()
        self.root.title("Sistema de Finanças Pessoais")
        self.root.geometry("1000x700")  # Aumentado para acomodar os gráficos
        self.criar_interface()

    def criar_interface(self):
        # Notebook para as abas
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill='both', expand=True, padx=10, pady=5)

        # Aba principal
        self.aba_principal = ttk.Frame(self.notebook)
        self.notebook.add(self.aba_principal, text='Transações')

        # Aba de gráficos
        self.aba_graficos = ttk.Frame(self.notebook)
        self.notebook.add(self.aba_graficos, text='Gráficos')

        self.criar_aba_principal()
        self.criar_aba_graficos()

    def criar_aba_principal(self):
        # Frame principal com grid weights
        main_frame = ttk.Frame(self.aba_principal)
        main_frame.grid(row=0, column=0, sticky='nsew', padx=10, pady=5)
        self.aba_principal.grid_columnconfigure(0, weight=1)
        self.aba_principal.grid_rowconfigure(0, weight=1)

        # Configurar grid weights para o main_frame
        main_frame.grid_columnconfigure(0, weight=1)
        main_frame.grid_rowconfigure(2, weight=1)  # A lista de transações deve expandir

        # Área de entrada de dados
        input_frame = ttk.LabelFrame(main_frame, text="Nova Transação", padding="10")
        input_frame.grid(row=0, column=0, sticky='ew', pady=5)

        # Grid para alinhar os campos de entrada
        for i in range(2):
            input_frame.grid_columnconfigure(i, weight=1)

        # Descrição
        ttk.Label(input_frame, text="Descrição:").grid(row=0, column=0, sticky='w', pady=2)
        self.descricao_entry = ttk.Entry(input_frame)
        self.descricao_entry.grid(row=0, column=1, sticky='ew', padx=5, pady=2)

        # Valor
        ttk.Label(input_frame, text="Valor (R$):").grid(row=1, column=0, sticky='w', pady=2)
        self.valor_entry = ttk.Entry(input_frame)
        self.valor_entry.grid(row=1, column=1, sticky='ew', padx=5, pady=2)

        # Categoria
        ttk.Label(input_frame, text="Categoria:").grid(row=2, column=0, sticky='w', pady=2)
        self.categoria_combo = ttk.Combobox(input_frame, values=self.sistema.categorias)
        self.categoria_combo.grid(row=2, column=1, sticky='ew', padx=5, pady=2)
        self.categoria_combo.set("Outros")

        # Tipo
        ttk.Label(input_frame, text="Tipo:").grid(row=3, column=0, sticky='w', pady=2)
        self.tipo_combo = ttk.Combobox(input_frame, values=["Receita", "Despesa"])
        self.tipo_combo.grid(row=3, column=1, sticky='ew', padx=5, pady=2)
        self.tipo_combo.set("Despesa")

        # Botão Adicionar
        ttk.Button(input_frame, text="Adicionar Transação", command=self.adicionar_transacao).grid(
            row=4, column=0, columnspan=2, pady=10, sticky='ew'
        )

        # Exibição do Saldo
        saldo_frame = ttk.LabelFrame(main_frame, text="Resumo", padding="10")
        saldo_frame.grid(row=1, column=0, sticky='ew', pady=5)
        
        self.saldo_label = ttk.Label(saldo_frame, text="", font=('Arial', 12, 'bold'))
        self.saldo_label.pack(expand=True)
        self.atualizar_saldo()

        # Lista de Transações
        lista_frame = ttk.LabelFrame(main_frame, text="Histórico de Transações", padding="10")
        lista_frame.grid(row=2, column=0, sticky='nsew', pady=5)

        # Configurar grid weights para o lista_frame
        lista_frame.grid_columnconfigure(0, weight=1)
        lista_frame.grid_rowconfigure(0, weight=1)

        # Treeview com scrollbar
        tree_frame = ttk.Frame(lista_frame)
        tree_frame.grid(row=0, column=0, sticky='nsew')
        tree_frame.grid_columnconfigure(0, weight=1)
        tree_frame.grid_rowconfigure(0, weight=1)

        self.tree = ttk.Treeview(tree_frame, columns=("Data", "Descrição", "Categoria", "Tipo", "Valor"))
        self.tree.heading("Data", text="Data")
        self.tree.heading("Descrição", text="Descrição")
        self.tree.heading("Categoria", text="Categoria")
        self.tree.heading("Tipo", text="Tipo")
        self.tree.heading("Valor", text="Valor")
        
        self.tree.column("#0", width=0, stretch=tk.NO)
        self.tree.column("Data", width=100)
        self.tree.column("Descrição", width=200)
        self.tree.column("Categoria", width=100)
        self.tree.column("Tipo", width=100)
        self.tree.column("Valor", width=100)

        # Scrollbars
        yscroll = ttk.Scrollbar(tree_frame, orient='vertical', command=self.tree.yview)
        xscroll = ttk.Scrollbar(tree_frame, orient='horizontal', command=self.tree.xview)
        self.tree.configure(yscrollcommand=yscroll.set, xscrollcommand=xscroll.set)

        # Grid
        self.tree.grid(row=0, column=0, sticky='nsew')
        yscroll.grid(row=0, column=1, sticky='ns')
        xscroll.grid(row=1, column=0, sticky='ew')

        # Botão Remover
        ttk.Button(lista_frame, text="Remover Transação Selecionada", command=self.remover_transacao).grid(
            row=1, column=0, pady=5, sticky='ew'
        )

        self.atualizar_lista()

    def criar_aba_graficos(self):
        # Frame principal para os gráficos
        frame_graficos = ttk.Frame(self.aba_graficos, padding="10")
        frame_graficos.grid(row=0, column=0, sticky='nsew')
        self.aba_graficos.grid_columnconfigure(0, weight=1)
        self.aba_graficos.grid_rowconfigure(0, weight=1)

        # Criar notebooks para diferentes tipos de gráficos
        notebook_graficos = ttk.Notebook(frame_graficos)
        notebook_graficos.pack(fill='both', expand=True)

        # Aba para gráfico de pizza
        aba_pizza = ttk.Frame(notebook_graficos)
        notebook_graficos.add(aba_pizza, text='Gráfico de Pizza')

        # Aba para gráfico de barras
        aba_barras = ttk.Frame(notebook_graficos)
        notebook_graficos.add(aba_barras, text='Gráfico de Barras')

        # Criar os gráficos
        self.criar_grafico_pizza(aba_pizza)
        self.criar_grafico_barras(aba_barras)

        # Botão para atualizar gráficos
        ttk.Button(frame_graficos, text="Atualizar Gráficos", command=self.atualizar_graficos).pack(pady=10)

    def criar_grafico_pizza(self, frame):
        figura = plt.Figure(figsize=(6, 4))
        self.canvas_pizza = FigureCanvasTkAgg(figura, frame)
        self.canvas_pizza.get_tk_widget().pack(fill='both', expand=True)
        self.ax_pizza = figura.add_subplot(111)
        self.atualizar_grafico_pizza()

    def criar_grafico_barras(self, frame):
        figura = plt.Figure(figsize=(6, 4))
        self.canvas_barras = FigureCanvasTkAgg(figura, frame)
        self.canvas_barras.get_tk_widget().pack(fill='both', expand=True)
        self.ax_barras = figura.add_subplot(111)
        self.atualizar_grafico_barras()

    def atualizar_grafico_pizza(self):
        self.ax_pizza.clear()
        despesas_por_categoria = defaultdict(Decimal)
        
        for transacao in self.sistema.transacoes:
            if transacao.tipo == "Despesa":
                despesas_por_categoria[transacao.categoria] += transacao.valor

        if despesas_por_categoria:
            categorias = list(despesas_por_categoria.keys())
            valores = list(despesas_por_categoria.values())
            self.ax_pizza.pie(valores, labels=categorias, autopct='%1.1f%%')
            self.ax_pizza.set_title('Distribuição de Despesas por Categoria')
        else:
            self.ax_pizza.text(0.5, 0.5, 'Sem dados para exibir', ha='center', va='center')

        self.canvas_pizza.draw()

    def atualizar_grafico_barras(self):
        self.ax_barras.clear()
        receitas_por_categoria = defaultdict(Decimal)
        despesas_por_categoria = defaultdict(Decimal)

        for transacao in self.sistema.transacoes:
            if transacao.tipo == "Receita":
                receitas_por_categoria[transacao.categoria] += transacao.valor
            else:
                despesas_por_categoria[transacao.categoria] += transacao.valor

        categorias = list(set(list(receitas_por_categoria.keys()) + list(despesas_por_categoria.keys())))
        receitas = [float(receitas_por_categoria[cat]) for cat in categorias]
        despesas = [float(despesas_por_categoria[cat]) for cat in categorias]

        x = range(len(categorias))
        largura = 0.35

        self.ax_barras.bar([i - largura/2 for i in x], receitas, largura, label='Receitas', color='green')
        self.ax_barras.bar([i + largura/2 for i in x], despesas, largura, label='Despesas', color='red')

        self.ax_barras.set_ylabel('Valor (R$)')
        self.ax_barras.set_title('Receitas e Despesas por Categoria')
        self.ax_barras.set_xticks(x)
        self.ax_barras.set_xticklabels(categorias, rotation=45, ha='right')
        self.ax_barras.legend()

        # Ajustar layout para evitar corte dos rótulos
        plt.tight_layout()
        self.canvas_barras.draw()

    def atualizar_graficos(self):
        self.atualizar_grafico_pizza()
        self.atualizar_grafico_barras()

    def adicionar_transacao(self):
        try:
            descricao = self.descricao_entry.get().strip()
            valor = self.valor_entry.get().strip()
            categoria = self.categoria_combo.get()
            tipo = self.tipo_combo.get()

            if not descricao or not valor:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos!")
                return

            valor = Decimal(valor.replace(",", "."))
            if valor <= 0:
                messagebox.showerror("Erro", "O valor deve ser maior que zero!")
                return

            transacao = TransacaoFinanceira(descricao, valor, categoria, tipo)
            self.sistema.adicionar_transacao(transacao)
            
            self.descricao_entry.delete(0, tk.END)
            self.valor_entry.delete(0, tk.END)
            self.categoria_combo.set("Outros")
            self.tipo_combo.set("Despesa")
            
            self.atualizar_lista()
            self.atualizar_saldo()
            self.atualizar_graficos()
            
        except ValueError:
            messagebox.showerror("Erro", "Valor inválido! Use apenas números e ponto/vírgula.")

    def remover_transacao(self):
        selecao = self.tree.selection()
        if not selecao:
            messagebox.showwarning("Aviso", "Selecione uma transação para remover!")
            return

        if messagebox.askyesno("Confirmar", "Deseja realmente remover esta transação?"):
            for item in selecao:
                indice = self.tree.index(item)
                self.sistema.remover_transacao(indice)
            
            self.atualizar_lista()
            self.atualizar_saldo()
            self.atualizar_graficos()

    def atualizar_lista(self):
        for item in self.tree.get_children():
            self.tree.delete(item)

        for transacao in self.sistema.transacoes:
            valor_formatado = f"R$ {transacao.valor:.2f}".replace(".", ",")
            self.tree.insert("", tk.END, values=(
                transacao.data,
                transacao.descricao,
                transacao.categoria,
                transacao.tipo,
                valor_formatado
            ))

    def atualizar_saldo(self):
        saldo = self.sistema.calcular_saldo()
        texto_saldo = f"Saldo atual: R$ {saldo:.2f}".replace(".", ",")
        self.saldo_label.configure(
            text=texto_saldo,
            foreground="green" if saldo >= 0 else "red"
        )

if __name__ == "__main__":
    root = tk.Tk()
    app = InterfaceGrafica(root)
    root.mainloop()

<Figure size 640x480 with 0 Axes>